In [3]:
from __future__ import unicode_literals
import youtube_dl
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
from dotenv import load_dotenv
import os
chrome_path = r"C:\Users\anves\Documents\chromedriver\chromedriver.exe"

In [4]:
#spotify setup
load_dotenv()
SPOTIFY_ID = os.getenv('SPOTIFY_ID')
SPOTIFY_SECRET = os.getenv('SPOTIFY_SECRET')
AUTH_URL = 'https://accounts.spotify.com/api/token'
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': SPOTIFY_ID,
    'client_secret': SPOTIFY_SECRET,
})
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}
spotify_base = 'https://api.spotify.com/v1/tracks/{id}'

In [5]:
def mp3download(link):
    ydl_opts = {
        'format': "bestaudio/best",
        'extractaudio':True,
        #'outtmpl': u'%(id)s.%(ext)s',
        'postprocessors':[{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'ffmpeg-location':r'ffmpeg.exe'
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([link])

In [6]:
def ytpull(song):
    ytbase = "https://www.youtube.com/results?search_query="
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(chrome_path, options=options)
    my_url = ytbase+song+"+lyrics"
    driver.get(my_url)

    user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
    link = user_data[0].get_attribute('href')
    return(link)

In [7]:
def spotifypull(uri):
    r = requests.get(spotify_base.format(id=uri), headers=headers)
    r = r.json()
    name = r['name']+" "+r['artists'][0]['name']
    return ytpull(name)

In [8]:
query = input("song: ")

#https://youtu.be/
#https://www.youtube.com/watch?v=
#"https://open.spotify.com/track/4npZ2Um2ZsNBzNOVEVCJoy?si=fwn5JNATRCikVo8XSAZScw"
#query = "spotify:track:4npZ2Um2ZsNBzNOVEVCJoy"

if query.find("https://") != -1:

    if query.find("youtube.com/watch") != -1 or query.find("youtu.be/") != -1:
        mp3download(query)

    if query.find("spotify") !=-1:
        uri = query[31:53]
        mp3download(spotifypull(uri))

elif query.find("spotify:track") !=-1:
    uri = query[14:]
    mp3download(spotifypull(uri))

else:
    mp3download(ytpull(query))

[youtube] DZXkDz80eRc: Downloading webpage
[download] Destination: Lil Tecca - Ransom Remix (Lyrics) ft. Juice Wrld-DZXkDz80eRc.webm
[download] 100% of 3.06MiB in 00:00                  
[ffmpeg] Destination: Lil Tecca - Ransom Remix (Lyrics) ft. Juice Wrld-DZXkDz80eRc.mp3
Deleting original file Lil Tecca - Ransom Remix (Lyrics) ft. Juice Wrld-DZXkDz80eRc.webm (pass -k to keep)
